In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import folium
import requests
from pathlib import Path
import os


In [2]:
# Configurations for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
plt.style.use('seaborn-darkgrid')

In [3]:
# Function to load data from API
def load_data_from_api(url):
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)

# Loading data from APIs (if used)
# Example API for traffic stops
traffic_stops_url = 'https://opendata.maryland.gov/resource/tx73-47dk.json'
traffic_stops_data = load_data_from_api(traffic_stops_url)



In [13]:
traffic_stops_data.info()
traffic_stops_data.columns
traffic_stops_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   year                                  41 non-null     object
 1   quarter                               41 non-null     object
 2   quarter_and_year                      41 non-null     object
 3   medevac_missions                      41 non-null     object
 4   impaired_driving_arrests              41 non-null     object
 5   cross_border_criminal_investigations  41 non-null     object
 6   total_traffic_stops                   41 non-null     object
dtypes: object(7)
memory usage: 2.4+ KB


,year,quarter,quarter_and_year,medevac_missions,impaired_driving_arrests,cross_border_criminal_investigations,total_traffic_stops
0,2014,Q1,Q1 2014,376,1496,180,110734
1,2014,Q2,Q2 2014,597,1630,104,141940
2,2014,Q3,Q3 2014,681,1767,148,147807
3,2014,Q4,Q4 2014,484,1562,108,119741
4,2015,Q1,Q1 2015,345,1500,210,132779


In [15]:
traffic_stops_data.tail()

,year,quarter,quarter_and_year,medevac_missions,impaired_driving_arrests,cross_border_criminal_investigations,total_traffic_stops
36,2023,Q1,Q1 2023,371,1256,320,98042
37,2023,Q2,Q2 2023,543,1241,333,87591
38,2023,Q3,Q3 2023,548,1163,294,80525
39,2023,Q4,Q4 2023,437,1158,243,79072
40,2024,Q1,Q1 2024,329,1211,299,92633


In [5]:
# Define the base directory
base_dir = Path('projectdata')

# Specific files
aadt_points_file = base_dir / 'Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Points).csv'
aadt_lines_file = base_dir / 'Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Lines).csv'
pedestrian_injury_file = base_dir / 'SHIP_Pedestrian_Injury_Rate_on_Public_Roads_2009-2022_20240505.csv'


In [6]:
if not aadt_points_file.exists():
    print(f"The file {aadt_points_file} does not exist!")
else:
    aadt_points_data = pd.read_csv(aadt_points_file)

if not aadt_lines_file.exists():
    print(f"The file {aadt_lines_file} does not exist!")
else:
    aadt_lines_data = pd.read_csv(aadt_lines_file)

if not pedestrian_injury_file.exists():
    print(f"The file {pedestrian_injury_file} does not exist!")
else:
    pedestrian_injury_data = pd.read_csv(pedestrian_injury_file)


C:\Users\aquil\AppData\Local\Temp/ipykernel_11048/2405018586.py:4: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_points_data = pd.read_csv(aadt_points_file)
C:\Users\aquil\AppData\Local\Temp/ipykernel_11048/2405018586.py:9: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_lines_data = pd.read_csv(aadt_lines_file)


In [7]:
aadt_points_data = pd.read_csv(aadt_points_file)
aadt_lines_data = pd.read_csv(aadt_lines_file)
pedestrian_injury_data = pd.read_csv(pedestrian_injury_file)

C:\Users\aquil\AppData\Local\Temp/ipykernel_11048/3372450109.py:1: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_points_data = pd.read_csv(aadt_points_file)
C:\Users\aquil\AppData\Local\Temp/ipykernel_11048/3372450109.py:2: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_lines_data = pd.read_csv(aadt_lines_file)


In [8]:
# General cleaning for all datasets
def clean_data(df):
    # Example: Convert dates, handle missing values
    df.dropna(inplace=True)
    return df

aadt_points_data = clean_data(aadt_points_data)
aadt_lines_data = clean_data(aadt_lines_data)
pedestrian_injury_data = clean_data(pedestrian_injury_data)


In [9]:
aadt_lines_data.info()
aadt_lines_data.columns
aadt_lines_data.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1222 entries, 3 to 4665
Columns: 102 entries, OBJECTID to Shape_Length
dtypes: float64(52), int64(11), object(39)
memory usage: 983.3+ KB


,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT_2010,AADT_2011,AADT_2012,AADT_2013,AADT_2014,AADT_2015,AADT_2016,AADT_2017,AADT_2018,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,Shape_Length0,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK_AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO,Shape_Length
3,4,B0810,2,Anne Arundel,0,None,John Hanson Hwy,IS,595,,2.64,2.15,6.930,Is595-.50 Mi E Of Md424,Md 424 To Is 97,Urban,1,Interstate,02000IS00595 01EE**************,02000IS00595--1-----,1,East,6,F,2,8.33,59.30,51.74,48.26,166.0099,https://maps.roads.maryland.gov/itms_public/?s...,85820.0,87580.0,87530.0,88321.0,88232.0,82640.0,90180.0,90190.0,94490.0,95151,90970.0,92840.0,92780.0,93621.0,92642.0,86770.0,94690.0,93800.0,98270.0,98961,260.0,75714.0,12222.0,679.0,2813.0,3463.0,NO ERROR,25131.428065,6276.0,0.448215,0.551785,0.002733,0.795725,0.128448,0.007136,0.029564,0.036395,0.065958,6,8.33,59.30,51.74,48.26,"85,820","87,580","87,530","88,321","88,232","82,640","90,180","90,190","94,490","95,151","90,970","92,840","92,780","93,621","92,642","86,770","94,690","93,800","98,270","98,961",260.0,"75,714",679.0,"12,222","2,813",3463.0,6276.0,9847.487806
7,8,B0743,2,Anne Arundel,0,None,Muddy Creek Rd,MD,468,,2.94,2.83,3.810,Md468-.10 Mi N Of Md256,Md 256 To Sudley Rd,Urban,5,Major Collector,02000MD00468 01NN**************,02000MD00468--1-----,1,North,2,C,0,8.42,66.81,50.05,49.95,3.5913,https://maps.roads.maryland.gov/itms_public/?s...,11221.0,11012.0,11580.0,10310.0,10301.0,10552.0,10370.0,10601.0,10542.0,10040,11781.0,11892.0,12280.0,11030.0,10921.0,11192.0,11100.0,11241.0,11182.0,10640,19.0,8375.0,1352.0,66.0,190.0,38.0,NO ERROR,5177.610588,228.0,0.833333,0.166667,0.001892,0.834163,0.134661,0.006574,0.018924,0.003785,0.022709,2,8.42,66.81,50.05,49.95,"11,221","11,012","11,580","10,310","10,301","10,552","10,370","10,601","10,542","10,040","11,781","11,892","12,280","11,030","10,921","11,192","11,100","11,241","11,182","10,640",19.0,"8,375",66.0,"1,352",190,38.0,228.0,2023.826824
10,11,B040004,4,Calvert,0,None,Dares Wharf Rd,MD,768,,0.10,0.00,0.680,Md768-.10 Mi N Of Md402,Md 402 To Chesapeake Ave,Urban,7,Local,04000MD00768 01EE**************,04000MD00768--1-----,1,North,2,F,0,9.99,64.94,48.38,51.62,0.1792,https://maps.roads.maryland.gov/itms_public/?s...,772.0,700.0,701.0,702.0,703.0,724.0,745.0,710.0,711.0,722,812.0,760.0,741.0,752.0,753.0,774.0,815.0,750.0,751.0,772,3.0,537.0,150.0,6.0,26.0,0.0,NO ERROR,3596.701014,26.0,1.000000,0.000000,0.004155,0.743767,0.207756,0.008310,0.036011,0.000000,0.036011,2,9.99,64.94,48.38,51.62,772,700,701,702,703,724,745,710,711,722,812,760,741,752,753,774,815,750,751,772,3.0,537,6.0,150,26,0.0,26.0,1403.170250
19,20,B030135,3,Baltimore,0,None,White Marsh Blvd,MD,43,,5.45,5.15,8.647,Md43-.30 Mi E Of Us40,Us 40 To Md 150,Urban,2,Principal Arterial Other Freeways and Expressways,03000MD00043 01EE**************,030

In [10]:
aadt_points_data.info()
aadt_points_data.columns
aadt_points_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 1 to 3017
Columns: 102 entries, X to TRUCK_AADT_ANNO
dtypes: float64(49), int64(11), object(42)
memory usage: 677.5+ KB


,X,Y,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT_2010,AADT_2011,AADT_2012,AADT_2013,AADT_2014,AADT_2015,AADT_2016,AADT_2017,AADT_2018,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK__AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO
1,-8.514937e+06,4.719226e+06,2,B0740,2,Anne Arundel,0,None,King George St,MD,450,,12.88,12.46,13.42,Md450-.10 Mi W Of College Ave,Main St To Md 435,Urban,4,Minor Arterial,02000MD00450 01EE**************,02000MD00450--1-----,1,East,2,F,2,7.36,68.52,48.39,51.61,3.6305,https://maps.roads.maryland.gov/itms_public/?s...,10991.0,11042.0,10390.0,10411.0,10382.0,9710.0,9891.0,10132.0,10360.0,10361,11761.0,11932.0,11220.0,11241.0,11112.0,10390.0,10581.0,10842.0,11090.0,10981,37.0,9258.0,881.0,31.0,146.0,8.0,NO ERROR,154.0,0.948052,0.051948,0.003571,0.893543,0.085030,0.002992,0.014091,0.000772,0.014863,2,7.36,68.52,48.39,51.61,"10,991","11,042","10,390","10,411","10,382","9,710","9,891","10,132","10,360","10,361","11,761","11,932","11,220","11,241","11,112","10,390","10,581","10,842","11,090","10,981",37.0,"9,258",31,881,146,8,154
2,-8.570495e+06,4.653970e+06,3,B1761,8,Charles,0,None,Port Tobacco Rd,MD,6,,24.75,22.95,25.21,Md6-.60 Mi W Of Us301,Chapel Point Rd To Us 301,Rural,5,Major Collector,08000MD00006 01EE**************,08000MD00006--1-----,1,West,2,F,1,8.99,51.54,50.08,49.92,8.2597,https://maps.roads.maryland.gov/itms_public/?s...,9740.0,9781.0,9702.0,9723.0,9694.0,9955.0,9860.0,10101.0,10012.0,10013,10420.0,10561.0,10482.0,10503.0,10374.0,10655.0,10550.0,10811.0,10712.0,10613,24.0,6915.0,2270.0,112.0,577.0,115.0,NO ERROR,692.0,0.833815,0.166185,0.002397,0.690602,0.226705,0.011185,0.057625,0.011485,0.069110,2,8.99,51.54,50.08,49.92,"9,740","9,781","9,702","9,723","9,694","9,955","9,860","10,101","10,012","10,013","10,420","10,561","10,482","10,503","10,374","10,655","10,550","10,811","10,712","10,613",24.0,"6,915",112,"2,270",577,115,692
5,-8.536933e+06,4.735299e+06,6,B0845,2,Anne Arundel,0,None,No Name,MD,32,,4.04,2.83,4.54,Md32-.50 Mi S Of Md170,Sappington Station Rd To Md 170,Urban,2,Principal Arterial Other Freeways and Expressways,02000MD00032 01NN**************,02000MD00032--1-----,1,South,4,C,1,8.59,68.24,47.80,52.20,32.7117,https://maps.roads.maryland.gov/itms_public/?s...,43070.0,43241.0,42902.0,44670.0,44541.0,45702.0,49720.0,50911.0,50452.0,52410,46090.0,46701.0,46332.0,48240.0,47661.0,48902.0,53200.0,54481.0,53982.0,55560,79.0,42093.0,6892.0,457.0,2174.0,715.0,NO ERROR,2889.0,0.752510,0.247490,0.001507,0.803148,0.131502,0.008720,0.041481,0.013642,0.055123,4,8.59,68.24,47.80,52.20,"43,070","43,241","42,902","44,670","44,541","45,702","49,720","50,911","50,452","52,410","46,090","46,701","46,332","48,240","47,661","48,902","53,200","54,481","53,982","55,560",79.0,"42,093",457,"6,892","2,174",715,"2,889"
9,-8.767689e+06,4.813219e+06,10,B010029,1,Alleg

In [11]:
pedestrian_injury_data.info()
pedestrian_injury_data.columns
pedestrian_injury_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 0 to 349
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Jurisdiction     268 non-null    object 
 1   Value            268 non-null    float64
 2   Race/ ethnicity  268 non-null    object 
 3   Year             268 non-null    int64  
 4   Measure          268 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 12.6+ KB


,Jurisdiction,Value,Race/ ethnicity,Year,Measure
0,State,53.5,All races/ ethnicities (aggregated),2017,Pedestrian Injury
2,Anne Arundel,37.3,All races/ ethnicities (aggregated),2017,Pedestrian Injury
3,Baltimore City,183.4,All races/ ethnicities (aggregated),2017,Pedestrian Injury
4,Baltimore County,54.4,All races/ ethnicities (aggregated),2017,Pedestrian Injury
7,Carroll,11.9,All races/ ethnicities (aggregated),2017,Pedestrian Injury


In [12]:
# Merging data examples
# Assuming common columns 'Date' and 'Location' for simplicity
combined_data = pd.merge(aadt_points_data, aadt_lines_data, on=['Location'], how='inner')
combined_data = pd.merge(combined_data, pedestrian_injury_data, on=['Date', 'Location'], how='inner')


KeyError: 'Location'

In [21]:
print("Columns in aadt_points_data:", aadt_points_data.columns)
print("Columns in aadt_lines_data:", aadt_lines_data.columns)
print("Columns in pedestrian_injury_data:", pedestrian_injury_data.columns)


Columns in aadt_points_data: Index(['X', 'Y', 'OBJECTID', 'LOCATION_ID', 'COUNTY_ID', 'COUNTY_DESC',
       'MUN_SORT', 'MUNICIPALITY', 'ROADNAME', 'ID_PREFIX',
       ...
       'AAWDT_2017_ANNO', 'AAWDT_2018_ANNO', 'AAWDT_ANNO',
       'MOTORCYCLE_AADT_ANNO', 'CAR_AADT_ANNO', 'BUS_AADT_ANNO',
       'LIGHT_TRUCK_AADT_ANNO', 'SINGLE_UNIT_AADT_ANNO',
       'COMBINATION_UNIT_AADT_ANNO', 'TRUCK_AADT_ANNO'],
      dtype='object', length=102)
Columns in aadt_lines_data: Index(['OBJECTID', 'LOCATION_ID', 'COUNTY_ID', 'COUNTY_DESC', 'MUN_SORT',
       'MUNICIPALITY', 'ROADNAME', 'ID_PREFIX', 'ID_RTE_NO', 'MP_SUFFIX',
       ...
       'AAWDT_2018_ANNO', 'AAWDT_ANNO', 'MOTORCYCLE_AADT_ANNO',
       'CAR_AADT_ANNO', 'BUS_AADT_ANNO', 'LIGHT_TRUCK_AADT_ANNO',
       'SINGLE_UNIT_AADT_ANNO', 'COMBINATION_UNIT_AADT_ANNO',
       'TRUCK_AADT_ANNO', 'Shape_Length'],
      dtype='object', length=102)
Columns in pedestrian_injury_data: Index(['Jurisdiction', 'Value', 'Race/ ethnicity', 'Year', 'Measu

In [ ]:
# Statistical summary and correlations
print(combined_data.describe())
sns.heatmap(combined_data.corr(), annot=True)
plt.show()


In [ ]:
# More complex analyses like regression, clustering, etc.
from sklearn.linear_model import LinearRegression

# Example: Linear regression to predict AADT from other factors
model = LinearRegression()
model.fit(combined_data[['Num_Lanes']], combined_data['AADT'])
predictions = model.predict(combined_data[['Num_Lanes']])


In [ ]:
# Geographic visualization of traffic data
map = folium.Map(location=[38.9072, -76.8569], zoom_start=10)
for idx, row in combined_data.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], radius=5, popup=str(row['AADT']), color='red').add_to(map)
map.save('Traffic_Map.html')


In [ ]:
# Summarize key findings and provide recommendations based on the analysis
print("Traffic volumes are highest on roads with X characteristic, suggesting Y policy interventions.")
